In [4]:
import csv
from collections import defaultdict
#import xlrd

#def parse_xlsx(f):
#    book = xlrd.open_workbook(f)
#    assert book.nsheets == 1
    
#    sh = book.sheet_by_index(0)
#    headers = [sh.cell_value(rowx=0, colx=i) for i in range(sh.ncols)]
#    res = {}
#    for i in range(sh.ncols):
#        res[headers[i]] = [sh.cell_value(rowx=j, colx=i)  for j in range(1, sh.nrows)]
#    
#    return res

def parse_csv(f):
    data = defaultdict(list)
    with open("/home/louis/code/hackqc2018/map/data/montrealnordarbrespublics20161207.csv") as fp:
        reader = csv.reader(fp)
        headers = next(reader)
        for row in reader:
            for h, c in zip(headers, row):
                data[h].append(c)

    return dict(data)
#         print(row)

In [52]:
import pandas as pd

dat = pd.read_csv("../data/arbres-publics.csv")

/home/danglewaggle/development/hackqc2018/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (4,5,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
dat.columns[9]

'Coord_Y'

In [56]:
from pymongo import MongoClient

In [57]:
conn = MongoClient()

In [58]:
db = conn['hackqc-2018']

In [35]:
client

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'hackqc-2018')

In [5]:
import os
from tqdm import tqdm
path = "../data/"
datas = {}
for file in tqdm(os.listdir(path)):
    try:
        file_path = "{}/{}".format(path, file)
        if file.endswith('.csv'):
            datas[file] = parse_csv(file_path)
        elif file.endswith('.xlsx'):
            datas[file] = parse_xlsx(file_path)
        else:
            raise ValueError(file)
    except Exception as e:
        print("Error on {}".format(file))
        print(e.__repr__())
    

100%|██████████| 1/1 [00:02<00:00,  2.46s/it]


In [19]:
from itertools import chain

keys = [list(d) for d in datas.values()][0]
res = {}

for k in tqdm(keys):
    res[k] = [e for d in datas.values() for e in d[k] ]



100%|██████████| 22/22 [00:00<00:00, 75.60it/s]


In [25]:
datas['arbres-publics.csv'].keys(

dict_keys(['INV_TYPE', 'EMP_NO', 'ARROND', 'ARROND_NOM', 'Rue', 'COTE', 'No_civique', 'Emplacement', 'Coord_X', 'Coord_Y', 'SIGLE', 'Essence_latin', 'Essence_fr', 'ESSENCE_ANG', 'DHP', 'Date_releve', 'Date_plantation', 'LOCALISATION', 'CODE_PARC', 'NOM_PARC', 'Longitude', 'Latitude'])

In [26]:
headers = list(res)

rows = [row for row in zip(*res.values()) if row[8] and row[9]]


In [27]:
res.keys()

dict_keys(['INV_TYPE', 'EMP_NO', 'ARROND', 'ARROND_NOM', 'Rue', 'COTE', 'No_civique', 'Emplacement', 'Coord_X', 'Coord_Y', 'SIGLE', 'Essence_latin', 'Essence_fr', 'ESSENCE_ANG', 'DHP', 'Date_releve', 'Date_plantation', 'LOCALISATION', 'CODE_PARC', 'NOM_PARC', 'Longitude', 'Latitude'])

In [22]:
set(res['Propriété'])


{'',
 '2002-05-30',
 '2002-06-12',
 '2002-09-06',
 '2004-08-19',
 '2004-12-01',
 '2005-08-11',
 '2012-09-25',
 'A déterminer',
 'Copropriété',
 'Mitoyen',
 'Privé',
 'Ville'}

In [8]:
from subprocess import Popen, PIPE
converter_js = "/../scripts/geographic-mtm-converter.js"

def convert(args):
    """ args is a list of triplet (w, h, zone)
        return a list of (lat,long)
    """
    payload = "".join("{}, {}, {}\n".format(*a) for a in args).encode('utf8')

    with Popen([node_path, converter_js], stdout=PIPE, stdin=PIPE) as proc:
        output = proc.communicate(input=payload)[0].decode('utf8').strip()
        return [[float(e) for e in line.split(',')] for line in output.split('\n')]



,INV_TYPE,EMP_NO,ARROND,ARROND_NOM,Rue,COTE,No_civique,Emplacement,Coord_X,Coord_Y,...,Essence_fr,ESSENCE_ANG,DHP,Date_releve,Date_plantation,LOCALISATION,CODE_PARC,NOM_PARC,Longitude,Latitude
0,H,6,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,287967.933,5043937.611,...,Févier Skyline,Skyline Honey-Locust,5.0,2004-06-10T00:00:00,2004-06-10T00:00:00,NaN,0005-000,RAIMBAULT,-73.715515,45.535151
1,H,7,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,287961.715,5043945.204,...,Févier Skyline,Skyline Honey-Locust,5.0,2004-06-10T00:00:00,2004-06-18T00:00:00,NaN,0005-000,RAIMBAULT,-73.715595,45.535219
2,H,9,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,287985.011,5043989.384,...,Frêne noir Fall Gold,Fall Gold Black Ash,5.0,2004-06-10T00:00:00,2004-06-18T00:00:00,NaN,0005-000,RAIMBAULT,-73.715298,45.535617
3,H,12,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,287979.931,5044012.360,...,Érable argenté 'Montréal',Montréal Siler Maple,5.0,2004-06-10T00:00:00,2004-06-18T00:00:00,NaN,0005-000,RAIMBAULT,-73.715364,45.535824
4,H,14,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,287965.452,5044070.727,...,Thuya occidental Nigra,Nigra American Arborvitae,5.0,2004-06-10T00:00:00,2004-06-18T00:00:00,NaN,0005-000,RAIMBAULT,-73.715551,45.536349
5,H,15,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,287967.522,5044063.667,...,Thuya occidental Nigra,Nigra American Arborvitae,5.0,2004-06-10T00:00:00,2004-06-18T00:00:00,NaN,0005-000,RAIMBAULT,-73.715525,45.536285
6,H,16,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,287960.871,5044065.611,...,Thuya occidental Nigra,Nigra American Arborvitae,5.0,2004-06-10T00:00:00,2004-06-18T00:00:00,NaN,0005-000,RAIMBAULT,-73.715610,45.536303
7,H,22,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,292748.613,5047926.130,...,Févier Skyline,Skyline Honey-Locust,20.0,2014-01-31T00:00:00,2004-07-06T00:00:00,NaN,0023-000,SAULT-AU-RÉCOLLET,-73.654402,45.571140
8,H,23,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,289568.014,5045285.446,...,Érable argenté 'Montréal',Montréal Siler Maple,5.0,2004-07-06T00:00:00,2004-07-06T00:00:00,NaN,0029-000,"MERCI, DE LA",-73.695070,45.547316
9,H,24,1,Ahuntsic - Cartierville,NaN,NaN,NaN,Parterre Gazonné,289548.539,5045258.457,...,Érable argenté 'Montréal',Montréal Siler Maple,19.0,2016-11-16T00:00:00,2004-07-06T00:00:00,NaN,0029-000,"MERCI, DE LA",-73.695318,45.547073


In [35]:
headers = ["ARROND", "ARROND_NOM", "Emplacement", "SIGLE", "Essence_latin", "Essence_fr", "ESSENCE_ANG",
           "DHP", "Date_releve", "Date_plantation", "NOM_PARC", "CODE_PARC"]

In [42]:
headers_lower = [h.lower() for h in headers]

In [44]:
a = [
    {'loc': list(lat_lon),
        **dict(zip(headers_lower, row))
    }
    for row, lat_lon in zip(dat[headers].values, dat[["Longitude", "Latitude"]].values)]

,INV_TYPE,EMP_NO,ARROND,ARROND_NOM,Rue,COTE,No_civique,Emplacement,Coord_X,Coord_Y,...,Essence_fr,ESSENCE_ANG,DHP,Date_releve,Date_plantation,LOCALISATION,CODE_PARC,NOM_PARC,Longitude,Latitude


In [48]:
a[20780]

{'loc': [nan, nan],
 'arrond': 4,
 'arrond_nom': 'Mercier - Hochelaga-Maisonneuve',
 'emplacement': 'Parterre Gazonné',
 'sigle': 'ACPLCO',
 'essence_latin': "Acer platanoides 'Columnare'",
 'essence_fr': 'Érable de Norvège colonnaire',
 'essence_ang': 'Columnare Norway Maple',
 'dhp': 50.0,
 'date_releve': '2014-11-11T00:00:00',
 'date_plantation': nan,
 'nom_parc': 'LOUIS-RIEL',
 'code_parc': '0495-000'}

In [4]:
import os

In [7]:
node_path = os.path.join(os.path.expanduser('~'), "versions/node/v8.11.1/bin/node")

In [9]:
convert(zip(dat["Coord_X"].values, dat["Coord_Y"].values, [8]*len(dat)))

FileNotFoundError: [Errno 2] No such file or directory: '/home/danglewaggle/versions/node/v8.11.1/bin/node': '/home/danglewaggle/versions/node/v8.11.1/bin/node'

In [37]:
convert()

['INV_TYPE',
 'EMP_NO',
 'ARROND',
 'ARROND_NOM',
 'Rue',
 'COTE',
 'No_civique',
 'Emplacement',
 'Coord_X',
 'Coord_Y',
 'SIGLE',
 'Essence_latin',
 'Essence_fr',
 'ESSENCE_ANG',
 'DHP',
 'Date_releve',
 'Date_plantation',
 'LOCALISATION',
 'CODE_PARC',
 'NOM_PARC',
 'Longitude',
 'Latitude']

In [103]:
from pymongo import MongoClient, GEO2D
import utm

db = MongoClient()['hackqc2018']
db.trees.create_index([("loc", GEO2D)])

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]


for chk_rows in tqdm(chunks(rows, 1000), total=len(rows)/1000):
    latlons = convert([(float(row[8]), float(row[9]), 8) for row in chk_rows])
    db.trees.insert_many([{
        'loc': latlon,
        **dict(zip(headers, row))
    } for row, latlon in zip(chk_rows, latlons)])


292it [00:53,  5.44it/s]                             


In [97]:
cursor = db.trees.find({'loc': {'$geoWithin': {
      '$box': [
          [-73.6007412, 45.5166326],
          [-73.582893, 45.524072]
      ]
  }}})

In [98]:
next(cursor)

{'_id': ObjectId('5aee05237d9b5343d29223cf'),
 'loc': [-73.600668, 45.520412],
 'arrond': 5,
 'arrond_nom': 'Le Plateau-Mont-Royal',
 'emplacement': 'Parterre avec Bordure',
 'sigle': 'ACPL',
 'essence_latin': 'Acer platanoides',
 'essence_fr': 'Érable de Norvège',
 'essence_ang': 'Norway Maple',
 'dhp': 19.0,
 'date_releve': '2017-03-15T00:00:00',
 'date_plantation': nan,
 'nom_parc': nan,
 'code_parc': nan}

In [99]:
 from enum import Enum
class Color(Enum):
    RED = 1
    GREEN = 2
    BLUE = 3

In [92]:
from urllib.request import urlretrieve
from urllib.parse import urlencode
mydict = {'latlon_sw': '45.5166326,-73.6007412', 'latlon_ne': '45.524072,-73.582893'}
qstr = urlencode(mydict)

In [93]:
qstr

'latlon_sw=45.5166326%2C-73.6007412&latlon_ne=45.524072%2C-73.582893'

In [116]:
from geojson import Feature, Point, FeatureCollection, dumps



cursor = db.trees.find({'loc': {'$geoWithin': {
    '$box': [
          [45.5166326,-73.6007412],
          [45.524072, -73.582893]
        ]
}}})

fc = FeatureCollection([Feature(geometry=Point(doc['loc'][::-1]), properties={
    'type': doc['Essence_Français'],
#     'age': doc['Date_plant']
}) for doc in tqdm(cursor)])


with open("/home/louis/code/hackqc2018/map/src/trees.js", 'w') as fp:
    fp.write("""module.exports = (()=>{
        return %s
    })()"""%dumps(fc))
    

1760it [00:00, 25984.54it/s]


In [43]:
TESTS = {
    (294551.328 ,   5044698.908): (45.54212936363354, -73.63123853240397),
    (296354.436, 5046722.007):(45.56035808507587, -73.60818394603155),
    (296352.045, 5046719.009):(45.56033107909284, -73.608214521844)
}

In [68]:
for mtm, latlon in TESTS.items():
    r = convert(*mtm, 8)
    

0.0 0.0
0.0 0.0
0.0 0.0


In [118]:
 from owslib.wms import WebMapService
wms = WebMapService('http://wms.jpl.nasa.gov/wms.cgi', version='1.1.1')
